In [ ]:
#%matplotlib notebook
%matplotlib inline
from matplotlib import rc
rc('figure',figsize=(9,4.5))
rc('font',size=12)
rc('text',usetex=False)

from qubicpack.qubicfp import qubicfp
import qubic.fibtools as ft
import qubic.plotters as p
import qubic.lin_lib as ll
import qubic.demodulation_lib as dl
import satorchipy as stpy
from pysimulators import FitsArray

import numpy as np
from matplotlib.pyplot import *
import matplotlib.mlab as mlab
import scipy.ndimage.filters as f
import glob
import string
import scipy.signal as scsig
from scipy import interpolate
import datetime as dt
import pickle
from importlib import reload

In [ ]:
day = '2019-11-08'
keyword = '*Test2_RHWP_fromQubicStudio*'
data_dir = '/qubic/Data/Calib-TD/'+day+'/'
dirs = np.sort(glob.glob(data_dir+keyword))
print (dirs)

voltage = np.array([2.])
fmod = np.array([1.])


In [ ]:
ifile = 0
thedir = dirs[ifile]
print(thedir)
a = qubicfp()
a.read_qubicstudio_dataset(thedir)
data=a.azel_etc(TES=None)
t0=data['t_data 1'][0]

hwp = a.hk['INTERN_HK']['HWP-Position']
hwp_timestamps = a.timeaxis(datatype='hk',axistype='computertime')

In [ ]:
print(a.hk.keys())
print(a.hk['INTERN_HK'].keys())
print(a.hk['CALSOURCE-CONF'].keys())

In [ ]:
data.keys()

In [ ]:
rc('figure',figsize=(9,4.5))
TESNum = 96
rc('figure',figsize=(12,8))
hwp_data = np.interp((data['t_data 1']-t0), (hwp_timestamps-hwp_timestamps[0]), hwp)

plot((data['t_data 1']-t0)/3600, dl.renorm(data['data 1'][TESNum-1,:]), 
     label='Data TES #{}'.format(TESNum))
xlabel('Hours')

plot((data['t_data 1']-t0)/3600, hwp_data, label='HWP Position')
legend()


In [ ]:
rc('figure',figsize=(13,8))
#### Filtering
reload(ft)
TESNum=96
tt = data['t_data 1']
t0 = tt[0]
dd = data['data 1'][TESNum-1,:]

thefreqmod = 1.

period = 1./ thefreqmod
lowcut = 0.8
highcut = 70

xmin = 0.01
xmax = 90.
ymin = 1e1
ymax = 1e17

############ Power spectrum
spectrum_f, freq_f = ft.power_spectrum(tt, dd, rebin=True)
plot(freq_f, f.gaussian_filter1d(spectrum_f,1),label='Raw Data')
yscale('log')
xscale('log')
xlabel('Frequency [Hz]')
ylabel('Power Spectrum')
xlim(xmin, xmax)
ylim(ymin, ymax)
title('TES#={}'.format(TESNum))

for i in range(10):
    plot([1./period*i,1./period*i],[ymin,ymax],'k--',alpha=0.3)

plot([lowcut, lowcut],[ymin,ymax],'k')
plot([highcut, highcut],[ymin,ymax],'k')
legend()

########## New Power spectrum
nharm = 10
notch = np.array([[1.724, 0.05, nharm]])
newdata = ft.filter_data(tt, dd, lowcut, highcut, notch=notch, rebin=True, verbose=True, order=5)
spectrum_f2, freq_f2 = ft.power_spectrum(tt, newdata, rebin=True)
plot(freq_f2, f.gaussian_filter1d(spectrum_f2,1),label='Filtered data')
for i in range(nharm):
    plot([notch[0,0]*(i+1),notch[0,0]*(i+1)], [ymin,ymax],'m:')
legend()

tight_layout()

In [ ]:
plot(tt, newdata)

In [ ]:
hwp_chunks = []
hwp_pos_chunk = []
istart = 0
hwp_pos = hwp_data[istart]
for i in range(len(hwp_data)):
    if hwp_data[i] == int(hwp_data[i]):
        if hwp_data[i] != hwp_data[istart]:
            hwp_chunks.append([istart+1500, i-1500])
            hwp_pos_chunk.append(int(hwp_data[istart]))
            istart = i
            
print(np.unique(np.array(hwp_pos_chunk)))

In [ ]:
rc('figure',figsize=(13,8))
ichunk = 0
chunk = hwp_chunks[ichunk]
tt = (data['t_data 1'][chunk[0]:chunk[1]]-t0)
dd = newdata[chunk[0]:chunk[1]]
subplot(2,1,1)
plot(tt,dd)
title('Pos={}'.format(hwp_pos_chunk[ichunk]))

subplot(2,1,2)
spectrum_f2, freq_f2 = ft.power_spectrum(tt, dd, rebin=True)
plot(freq_f2, f.gaussian_filter1d(spectrum_f2,1),label='Filtered data')
for i in range(nharm):
    plot([notch[0,0]*(i+1),notch[0,0]*(i+1)], [ymin,ymax],'m:')
yscale('log')
xscale('log')
xlabel('Frequency [Hz]')
ylabel('Power Spectrum')
xlim(xmin, xmax)
ylim(ymin, ymax)


In [ ]:
rc('figure',figsize=(13,8))
cols = ['r','b','g','k','m','y','orange']
angles = np.arange(7)*15
for i in range(len(hwp_chunks)):
    chunk = hwp_chunks[i]
    plot((data['t_data 1'][chunk[0]:chunk[1]]-t0)/3600, newdata[chunk[0]:chunk[1]], 
        color=cols[hwp_pos_chunk[i]-1])

xlim(0,np.max((data['t_data 1']-t0)/3600))
mm,ss = ft.meancut(newdata,3)
ylim(mm-2*ss,mm+3*ss)
for i in arange(1,8):
    plot(-1,-1, 'o',color=cols[i-1], label='HWP Pos = {} ; Angle = {} deg.'.format(i, angles[i-1]))
legend()

In [ ]:
#### RMS Method - Not so good as all the noise goes into the HWP modulation signal, so this is just an upper limit
# TESNum = 96
# tt = data['t_data 1']
# t0 = tt[0]
# dd = data['data 1'][TESNum-1,:]
# newdata = ft.filter_data(tt, dd, lowcut, highcut, notch=None, rebin=True, verbose=True)

# thvals = np.zeros(len(hwp_chunks))
# ampvals = np.zeros(len(hwp_chunks))
# ampvals_err = np.zeros(len(hwp_chunks))
# for i in range(len(hwp_chunks)):
#     chunk = hwp_chunks[i]
#     thvals[i] = angles[hwp_pos_chunk[i]-1]
#     thedata = newdata[chunk[0]:chunk[1]]
#     ampvals[i] = np.std(thedata)
#     ampvals_err[i] = ampvals[i] / 2. / np.sqrt(len(thedata))

In [ ]:
rc('figure',figsize=(16,3))
#### Sine Fitting Method
def sinewave(x, pars, extra_args=None):
    offset = pars[0]
    amplitude = pars[1]
    phase = pars[2]
    modfreq = pars[3]
    return(offset+amplitude*np.sin(2*np.pi*x*modfreq+phase))

TESNum = 96
tt = data['t_data 1']
t0 = tt[0]
dd = data['data 1'][TESNum-1,:]
newdata = ft.filter_data(tt, dd, lowcut, highcut, notch=None, rebin=True, verbose=True)

thvals = np.zeros(len(hwp_chunks))
ampvals = np.zeros(len(hwp_chunks))
ampvals_err = np.zeros(len(hwp_chunks))
for i in range(len(hwp_chunks)):
    print(i, len(hwp_chunks))
    chunk = hwp_chunks[i]
    thvals[i] = angles[hwp_pos_chunk[i]-1]
    thedata = newdata[chunk[0]:chunk[1]]
    thett = tt[chunk[0]:chunk[1]]
    #thedata = thedata[500:1000]
    #thett = thett[500:1000]
    guess = np.array([np.std(thedata), 0., 0., 1.])
    fitsine = ft.do_minuit(thett, thedata, thedata*0+1., guess, functname=sinewave,
              force_chi2_ndf=False, fixpars=[0,0,0,1], verbose=False)
    figure()
    plot(thett, thedata,'.')
    plot(thett, sinewave(thett, fitsine[1]))
    title('HWP Pos = {} ; Angle = {} deg.'.format(hwp_pos_chunk[i], angles[hwp_pos_chunk[i]-1]))
    xlabel('Time [s]')
    ylabel('ADU')
    print(fitsine[1])
    print(fitsine[2])
    print('')
    ampvals[i] = fitsine[1][1]
    ampvals_err[i] = fitsine[2][1]


In [ ]:
rc('figure',figsize=(13,8))

def hwp_sin(x, pars, extra_args=None):
    amplitude = pars[0]
    XPol = 1-pars[1]
    phase = pars[2]
    return(amplitude*0.5*(1-np.abs(XPol)*np.sin(4*np.radians(x+phase))))

# def hwp_sin(x, pars, extra_args=None):
#     amplitude = pars[0]
#     XPol = np.abs(pars[1])
#     fact=(1.0-XPol)#/(1.0+XPol)
#     phase = pars[2]
#     return(amplitude*0.5*(1.0 + fact*np.sin(4*np.radians(x+phase))))

okdata = ampvals_err != 0
guess = np.array([np.max(ampvals)*2, 0, 0.])
fithwp = ft.do_minuit(thvals[okdata], np.abs(ampvals[okdata]), ampvals_err[okdata], guess, functname=hwp_sin,
              force_chi2_ndf=True, verbose=False)
print('Fitted')
print(fithwp[1])
print('Errors')
print(fithwp[2])
errorbar(thvals[okdata], np.abs(ampvals[okdata])/fithwp[1][0], yerr= ampvals_err[okdata]/fithwp[1][0], fmt='r.')
angs = np.linspace(0,90,90)
plot(angs, hwp_sin(angs, fithwp[1])/fithwp[1][0], 
     label='XPol = {2:5.2f}% +/- {3:5.2f}% '.format(fithwp[1][0], fithwp[2][0], 
                                                fithwp[1][1]*100, fithwp[2][1]*100,
                                                fithwp[1][2], fithwp[2][2]))
plot(angs, angs*0+0.5,'k:')
ylim(0,1)
legend()